In [2]:
from datasets import load_dataset
data = load_dataset('cnn_dailymail', '3.0.0', split = 'train[:50000]')
data

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 50000
})

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = 't5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

2025-05-30 07:29:13.930926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748590154.115970      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748590154.167298      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [4]:
def preprocess_function(text):
  inputs = ["summarize: " + article for article in text["article"]]
  model_input = tokenizer(inputs, max_length = 1024, truncation = True, padding = 'max_length', return_tensors = 'pt')
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(text["highlights"], max_length=254, truncation=True, padding="max_length")
  model_input["labels"] = labels["input_ids"]
  return model_input

tokenized_dataset = data.map(preprocess_function, batched = True, desc="Tokenizing the dataset")

Tokenizing the dataset:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [5]:
split_dataset = tokenized_dataset.train_test_split(test_size = 0.2)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [6]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654


In [7]:
!pip install rouge_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=62927e913d3e38dac3dc6578f69484cb74ec56c5dfea4b0e860e3a38bbb3e440
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [8]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
import evaluate
rouge = evaluate.load("rouge")
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return {
        "rouge1": result["rouge1"].mid.fmeasure,
        "rouge2": result["rouge2"].mid.fmeasure,
        "rougeL": result["rougeL"].mid.fmeasure,
    }

In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-lora-summarization",
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    logging_dir="./logs",
    logging_steps=1000,
    report_to="none",
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model.base_model.config.label_names = ["labels"]
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

train_result = trainer.train()
print(train_result)

/tmp/ipykernel_35/500979434.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1000,0.964500
2000,0.530900
3000,0.520300
4000,0.517000
5000,0.518400
6000,0.515000
7000,0.515400
8000,0.515400
9000,0.511600
10000,0.513500


TrainOutput(global_step=15000, training_loss=0.5451429972330729, metrics={'train_runtime': 6390.0836, 'train_samples_per_second': 18.779, 'train_steps_per_second': 2.347, 'total_flos': 3.2916897792e+16, 'train_loss': 0.5451429972330729, 'epoch': 3.0})


In [11]:
model.save_pretrained("./t5-lora-summarization/adapter")
tokenizer.save_pretrained("./t5-lora-summarization/tokenizer")
model.base_model.save_pretrained("./t5-lora-summarization/base_model")


In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel, PeftConfig

tokenizer = T5Tokenizer.from_pretrained("./t5-lora-summarization/tokenizer")

base_model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = PeftModel.from_pretrained(base_model, "./t5-lora-summarization/adapter")

model.eval()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
             

In [13]:
def summarize(text):
    input_ids = tokenizer("summarize: " + text, return_tensors="pt", padding=True, truncation=True).input_ids
    input_ids = input_ids.to(model.device)  # ensure it's on the correct device
    output_ids = model.base_model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

article = "Chandrayaan-1, India's first mission to Moon, was launched successfully on October 22, 2008 from SDSC SHAR, Sriharikota. The spacecraft was orbiting around the Moon at a height of 100 km from the lunar surface for chemical, mineralogical and photo-geologic mapping of the Moon. The spacecraft carried 11 scientific instruments built in India, USA, UK, Germany, Sweden and Bulgaria."

article = article + "After the successful completion of all the major mission objectives, the orbit has been raised to 200 km during May 2009. The satellite made more than 3400 orbits around the moon and the mission was concluded when the communication with the spacecraft was lost on August 29, 2009."
print(summarize(article))


Chandrayaan-1, India's first mission to Moon, launched successfully on October 22, 2008. The spacecraft was orbiting around the Moon at a height of 100 km from the lunar surface for chemical, mineralogical and photo-geologic mapping of the Moon.


In [23]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from peft import PeftModel
import torch

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("./t5-lora-summarization/tokenizer")
base_model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = PeftModel.from_pretrained(base_model, "./t5-lora-summarization/adapter")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()


def summarize(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    with torch.no_grad():
        output_ids = model.base_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=4,
            early_stopping=True
        )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [24]:
summarize(article)

"Chandrayaan-1, India's first mission to Moon, launched successfully on October 22, 2008. The spacecraft was orbiting around the Moon at a height of 100 km from the lunar surface for chemical, mineralogical and photo-geologic mapping of the Moon."

In [32]:
from tqdm import tqdm
model.eval()

generated_summaries = []
reference_summaries = []

# Loop through test dataset with progress bar
for item in tqdm(test_dataset.select(range(1000)), desc="Evaluating"):
    # Convert to tensors
    input_ids = torch.tensor(item["input_ids"]).unsqueeze(0).to(model.device)
    label_ids = torch.tensor(item["labels"])

    # Generate prediction
    output_ids = model.base_model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)
    pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    ref = tokenizer.decode(label_ids, skip_special_tokens=True)

    generated_summaries.append(pred)
    reference_summaries.append(ref)

# Compute and print ROUGE scores
results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
print("ROUGE-1:", results["rouge1"])
print("ROUGE-2:", results["rouge2"])
print("ROUGE-L:", results["rougeL"])


Evaluating: 100%|██████████| 1000/1000 [15:47<00:00,  1.06it/s]


ROUGE-1: 0.3402354156965203
ROUGE-2: 0.1461615747582956
ROUGE-L: 0.24869420870858897
